# UFC_Stats Web Scraping

## importing pertinent libraries

In [1]:
#importing pertinent libraries
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests, re
import json
import itertools
from datetime import datetime
import pickle
import numpy as np

In [2]:
#creating datetime object to parse out future events

today = datetime.today()
current_date= today.strftime("%B %d, %Y")
current_datetime = datetime.strptime(current_date, "%B %d, %Y" ) 

## Fetching Fighter Url List

In [3]:
with open('fighter_url_list', 'rb') as f:
    fighter_url_list = pickle.load(f)

In [4]:
fighter_url_list[:5]

['http://www.ufcstats.com/fighter-details/ee0b69e307c857e5',
 'http://www.ufcstats.com/fighter-details/269d103c96a4c3a5',
 'http://www.ufcstats.com/fighter-details/8753e125f4499816',
 'http://www.ufcstats.com/fighter-details/0e9869d712e81f8f',
 'http://www.ufcstats.com/fighter-details/3738e68d2261e60f']

## Creating Dataframe Building Blocks

### Generating basic statistics lists

In [231]:
def soup_generator(url):
    '''Meant to parse any url via beuatiful soup
    
    Args:
        url(str): url that will be parsed using BeautifulSoup
        '''
    
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    return soup

In [232]:
#creating a soup list to reduce number of website requests
fighter_url_soup_list = list(map(soup_generator, fighter_url_list))

In [233]:
def base_stats_soup_generator(soup):
    '''Meant to identiy statistics in the context of UFC Stats urls.
    
    Args:
        
        soup(BeautifulSoup object) : must be an unaltered soup'''
    
    base_stats = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    
    return base_stats

In [234]:
fighter_stats_objects = list(map(base_stats_soup_generator, fighter_url_soup_list))

### Construction of Fighter Physical Statistics DataFrame 

In [78]:
def fighter_name(soup):
    '''Identifies fighter history to be parsed.
    Args:
    
        soup(BeautifulSoup object): BeautifulSoup object must originate from a urls hosting fighter profiles'''
    
    name = soup.find('span', attrs={'class': re.compile('b-content__title-highlight')})
    
    return str(name.text.strip())

In [82]:
def fighter_record(soup):
    
    record = soup.findChildren('span', attrs={'class': re.compile('b-content__title-record')})
    
    return record[0].text.strip().split(':')[1].strip()

In [83]:
def basic_stats_height(soup):
    
    '''soups run thorugh this function should be run through stats_soup function beforehand.'''
    
    #feet
    height = soup[0].text.strip().split(':')[1].strip().split(' ')[0].replace("'", "")
    
   #inches
    if height == '--':
        
        pass
    
    else:
        
        height = int(height)
        
        inches = int(soup[0].text.strip().split(':')[1].strip().split(' ')[1].replace("'", "").replace('"', '').replace("--", "0"))

        height_to_inches = (height * 12) 
    
        final_height = height_to_inches + inches
    
        return int(final_height)

In [111]:
def basic_stats_weight(soup):
    
    weight = soup[1].text.strip().split(':')[1].strip().replace(' lbs.', '')
    
    if weight == '--':
        
        pass
    
    else:
        
        weight = int(weight)
        
    return weight

In [98]:
def basic_stats_reach(soup):
    
    reach = soup[2].text.strip().split(':')[1].strip().replace('"', '')
    
    if reach == '--':
        
        pass
    
    else:
        
        reach = int(reach)
        
    return reach

In [94]:
def basic_stats_stance(soup):
    
    stance = soup[3].text.strip().split(':')[1].strip()
    
    return str(stance)

In [100]:
def basic_stats_DOB(soup):
    
    DOB = soup[4].text.strip().split(':')[1].strip()
    
    return DOB

In [113]:
fighter_names = list(map(fighter_name, fighter_url_soup_list))

fighter_records = list(map(fighter_record, fighter_url_soup_list))

fighter_heights =  list(map(basic_stats_height, fighter_stats_objects))

fighter_weights = list(map(basic_stats_weight, fighter_stats_objects))

fighter_reachs = list(map(basic_stats_reach, fighter_stats_objects))

fighter_stances = list(map(basic_stats_stance, fighter_stats_objects))

fighter_DOBs = list(map(basic_stats_DOB, fighter_stats_objects))

In [114]:
fighter_physical_stats_df = pd.DataFrame()

fighter_physical_stats_df['name'] = fighter_names

fighter_physical_stats_df['record'] = fighter_records

fighter_physical_stats_df['height'] = fighter_heights

fighter_physical_stats_df['weight'] = fighter_weights

fighter_physical_stats_df['reach'] = fighter_reachs

fighter_physical_stats_df['stance'] = fighter_stances

fighter_physical_stats_df['DOB'] = fighter_DOBs

In [115]:
fighter_physical_stats_df.head()

,name,record,height,weight,reach,stance,DOB
0,Chris Amarante,2-0-0,NaN,185,--,,--
1,Kenji Arai,15-15-5,70.0,145,--,Southpaw,"Oct 05, 1979"
2,Blas Avena,8-7-0 (1 NC),72.0,170,74,Orthodox,"Jun 30, 1983"
3,Sam Adkins,7-20-2,75.0,225,--,Orthodox,"Apr 26, 1965"
4,Andrei Arlovski,29-19-0 (2 NC),75.0,240,77,Orthodox,"Feb 04, 1979"


In [105]:
fighter_physical_stats_df.to_csv('fighter_physical_stats_csv', index=False)

### Construction of Career Statistics DataFrame 

In [5]:
def career_stats_soup(soup):
    
    career_stats_soup = soup.findChildren('li', attrs={'class': re.compile('b-list')})
    
    career_stats = career_stats_soup[5:]
    
    return career_stats_soup

In [6]:
def career_stats_SLpM(soup):

    SLpM = soup[5].text.split(':')[1].replace('%', '').strip()
    
    if SLpM == '--':
        
        pass
    
    else:
        
        SLpM = float(SLpM)
        
    return SLpM

In [7]:
def career_stats_StrAcc(soup):
    
    StrAcc =  soup[6].text.split(':')[1].replace('%', '').strip()
    
    if StrAcc == '--':
        
        pass
    
    else:
        
        StrAcc = float(StrAcc)
        
    return StrAcc

In [8]:
def career_stats_SApM(soup):
    
    SApM = soup[7].text.split(':')[1].strip()
    
    if SApM == '--':
        
        pass
    
    else:
        
        SApM = float(SApM)
        
    return SApM

In [9]:
def career_stats_StrDef(soup):
    
    StrDef = soup[8].text.split(':')[1].replace('%', '').strip()
    
    if StrDef == '--':
        
        pass
    
    else:
        
        StrDef = float(StrDef)
        
    return StrDef

In [10]:
def career_stats_TDAvg(soup):
    
    TDAvg = soup[10].text.split(':')[1].strip()
    
    if TDAvg == '--':
        
        pass
    
    else:
        
        TDAvg = float(TDAvg)
        
    return TDAvg

In [11]:
def career_stats_TDAcc(soup):
    
    TDAcc = soup[11].text.split(':')[1].strip().replace('%', '')
    
    if TDAcc == '--':
        
        pass
    
    else:
        
        TDAcc = float(TDAcc)
        
    return TDAcc

In [12]:
def career_stats_TDDef(soup):
    
    TDDef = soup[12].text.split(':')[1].replace('%', '').strip()
    
    if TDDef == '--':
        
        pass
    
    else:
        
        TDDef = float(TDDef)
        
    return TDDef

In [13]:
def career_stats_SubAvg(soup):
    
    SubAvg =  soup[13].text.split(':')[1].strip().replace('%', '')  
    
    if SubAvg == '--':
        
        pass
    
    else:
        
        SubAvg = float(SubAvg)
        
    return SubAvg

In [24]:
fighter_SLpMs = list(map(career_stats_SLpM, fighter_stats_objects))

fighter_StrAccs = list(map(career_stats_StrAcc, fighter_stats_objects))

fighter_SApMs = list(map(career_stats_SApM, fighter_stats_objects))

fighter_StrDefs =list(map(career_stats_StrDef, fighter_stats_objects))

fighter_TDAvgs = list(map(career_stats_TDAvg, fighter_stats_objects))

fighter_TDAccs = list(map(career_stats_TDAcc, fighter_stats_objects))

fighter_TDDefs = list(map(career_stats_TDDef, fighter_stats_objects))

fighter_SubAvgs = list(map(career_stats_SubAvg, fighter_stats_objects))

In [34]:
fighter_career_stats_df = pd.DataFrame()

fighter_career_stats_df['name'] = fighter_names

fighter_career_stats_df['SLpMs'] = fighter_SLpMs

fighter_career_stats_df['StrAccs'] = fighter_StrAccs

fighter_career_stats_df['SApMs'] = fighter_SApMs

fighter_career_stats_df['StrDefs'] = fighter_StrDefs

fighter_career_stats_df['TDAvgs'] = fighter_TDAvgs

fighter_career_stats_df['TDAccs'] = fighter_TDAccs

fighter_career_stats_df['TDDefs'] = fighter_TDDefs

fighter_career_stats_df['SubAvgs'] = fighter_SubAvgs

In [35]:
fighter_career_stats_df.head()

,name,SLpMs,StrAccs,SApMs,StrDefs,TDAvgs,TDAccs,TDDefs,SubAvgs
0,Chris Amarante,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
1,Kenji Arai,8.79,27.0,10.29,39.0,0.00,0.0,0.0,0.0
2,Blas Avena,2.34,47.0,3.36,43.0,2.37,80.0,50.0,1.8
3,Sam Adkins,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,Andrei Arlovski,3.54,44.0,2.69,59.0,0.50,36.0,79.0,0.3


In [46]:
fighter_career_stats_df.to_csv('fighter_career_stats_csv', index=False)

### Constructing Fight Event and Fight List Database

In [9]:
with open('event_url_list', 'rb') as f:
    event_url_list = pickle.load(f)

In [10]:
event_url_list[:5]

['http://www.ufcstats.com/event-details/c0231720fe516994',
 'http://www.ufcstats.com/event-details/aa5b4eff51bdc7d1',
 'http://www.ufcstats.com/event-details/32a3025d5db456ae',
 'http://www.ufcstats.com/event-details/b3fb8d2293e17a59',
 'http://www.ufcstats.com/event-details/bad28b7b34f334de']

#### Event Database Populator

In [317]:
def fighter_column_transformer(df):
    
    '''This function reformats df['Fighter'] into first/last name pairs for fighters
    
    Args:
    
    df:
    
    column: target columne intended to be transformed into a list '''
    
    final_pairs = []
    
    for x in range(0, len(df.index)):

        value_split = df.iloc[x]['Fighter'].split()

        split_values = [value_split[index : index + 2] for index in range(0, len(value_split), 2)]

        column_value_pair = [' '.join(split_values[x]) for x in range(0,len(split_values))]
        
        final_pairs.append(column_value_pair)
        
    df.Fighter = final_pairs

In [318]:
def column_transformer(df, column):
    
    '''This function reformats any df[column] with 2 item sublists in individual integers

    Args:
    df:
    column: target columne intended to be transformed into a list '''
    
    value_pairs = []
    
    for x in range(0, len(df.index)):
        
        values_split = df.iloc[x][column].split()
        
        values_split = [value for value in values_split]
        
        value_pairs.append(values_split)
        
    df[column] = value_pairs

In [319]:
def event_name_fetcher(event_soup):
    event_title = event_soup.find('h2', attrs={'class': re.compile('b-content__')}).text.strip()
    return event_title

In [320]:
def event_date_fetcher(event_soup):
    date_holder = event_soup.findChildren('li', attrs={'class': re.compile('b-list__')})
    
    try:
        date = date_holder[0].text.split(':')[1].strip()
        
        return date 
        
    except:
        
        pass    

In [321]:
def unnesting(df, explode):
    idx = df.index.repeat(df[explode[0]].str.len())
    df1 = pd.concat([pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
    df1.index = idx
    
    return df1.join(df.drop(explode, 1), how='left')

In [322]:
test_frame_urls = event_url_list[:5]

In [323]:
test_frames = [pd.read_html(url)[0] for url in test_frame_urls]

In [324]:
target_columns = ['Str', 'Td', 'Sub', 'Pass']
transformed_df = []


for url in test_frame_urls[:2]:
    event_soup = soup_generator(url)
    frame_holder = pd.read_html(url)[0]   
    frame_holder['Event'] = event_name_fetcher(event_soup)
    frame_holder['Date'] = event_date_fetcher(event_soup)
    
    fighter_column_transformer(frame_holder)
    
    for column in target_columns:
        column_transformer(frame_holder, column)
           
    transformed_df.append(unnesting(frame_holder, ['Fighter', 'Str', 'Td', 'Sub', 'Pass']))

In [325]:
transformed_df = pd.concat(transformed_df)

In [326]:
transformed_df

,Fighter,Str,Td,Sub,Pass,W/L,Weight class,Method,Round,Time,Event,Date
0,Jimi Manuwa,6,0,0,0,win,Light Heavyweight,KO/TKO Punch,1,3:05,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
0,Corey Anderson,6,0,0,0,win,Light Heavyweight,KO/TKO Punch,1,3:05,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
1,Gunnar Nelson,8,1,1,5,win,Welterweight,SUB Guillotine Choke,2,0:46,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
1,Alan Jouban,17,0,0,0,win,Welterweight,SUB Guillotine Choke,2,0:46,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
2,Marlon Vera,67,0,0,0,win,Catch Weight,KO/TKO Kick,3,3:50,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
2,Brad Pickett,51,2,0,2,win,Catch Weight,KO/TKO Kick,3,3:50,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
3,Arnold Allen,32,3,1,6,win,Featherweight,S-DEC,3,5:00,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
3,Makwan Amirkhani,17,4,3,3,win,Featherweight,S-DEC,3,5:00,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
4,Joe Duffy,51,0,0,2,win,Lightweight,U-DEC,3,5:00,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"
4,Reza Madadi,28,2,0,0,win,Lightweight,U-DEC,3,5:00,UFC Fight Night: Manuwa vs. Anderson,"March 18, 2017"


***************************************************************************************************************